In [ ]:
from imutils import contours
import imutils
import cv2 
import numpy as np
import torch
import cv2
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from pandas.core.common import flatten
from torchvision import datasets, transforms, models
import PIL
from PIL import Image
import torchvision.transforms.functional as TF
import random
import re

In [ ]:
class Network(nn.Module):
	def __init__(self):
		super(Network,self).__init__()
		self.cl1=nn.Conv2d(1,16,3,padding=1)
		self.cl1_bn=nn.BatchNorm2d(16)
		self.cl2=nn.Conv2d(16,32,3,padding=1)
		self.cl2_bn=nn.BatchNorm2d(32)
		self.cl3=nn.Conv2d(32,64,3,padding=1)
		self.cl3_bn=nn.BatchNorm2d(64)
		self.dropout=nn.Dropout(0.2)
		self.fc1=nn.Linear(4096,1024)
		self.fc1_bn=nn.BatchNorm1d(1024)
		self.fc2=nn.Linear(1024,256)
		self.fc2_bn=nn.BatchNorm1d(256)
		self.fc3=nn.Linear(256,14)
		self.pool=nn.MaxPool2d(2,2)

	def forward(self,x):
		x=self.cl1(x)
		x=self.pool(F.leaky_relu(self.cl1_bn(x)))
		x=self.cl2(x)
		x=self.pool(F.leaky_relu(self.cl2_bn(x)))
		x=self.cl3(x)
		x=self.pool(F.leaky_relu(self.cl3_bn(x)))
		x=x.view(-1,4096)
		self.dropout(x)
		x=self.fc1(x)
		x=F.relu(self.fc1_bn(x))
		self.dropout(x)
		x=self.fc2(x)
		x=F.relu(self.fc2_bn(x))
		self.dropout(x)
		x=self.fc3(x)
		return x

model=Network()
model.cuda()

In [ ]:
model.load_state_dict(torch.load('/home/legendarygene/Desktop/Handwritten Arithmetic Calculator/model_ERA_lr0005.pt'))
model.eval()

In [ ]:
transform_test = transforms.Compose([transforms.Resize((64,64)),
									 transforms.Grayscale(),
									 transforms.ToTensor(),
									 transforms.Normalize((0.5), (0.5))])

In [ ]:
classes=['+','-','0','1','2','3','4','5','6','7','8','9','/','*']

In [ ]:
path="/home/legendarygene/Desktop/Handwritten Arithmetic Calculator/Final Testing/image.jpeg"
image = cv2.imread(path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_,img=cv2.threshold(gray,150,255,cv2.THRESH_BINARY_INV)
cnts = cv2.findContours(img, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
digitCnts = []

for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    digitCnts.append(c)
digitCnts = contours.sort_contours(digitCnts,method="left-to-right")[0]
symbols = []
to_calculate = []

for c in digitCnts:
    (x, y, w, h) = cv2.boundingRect(c)
    roi = img[y:y + h, x:x + w]
    _,roi = cv2.threshold(roi,200,255,cv2.THRESH_BINARY_INV)
    index = 0
    if roi.shape[0] > roi.shape[1]:
        index = roi.shape[0]
        diff = roi.shape[0] - roi.shape[1]
        roi = cv2.copyMakeBorder(roi, 0, 0, diff//2, diff//2,cv2.BORDER_CONSTANT ,value=255)
    elif roi.shape[1] > roi.shape[0]:
        index = roi.shape[1]
        diff = roi.shape[1] - roi.shape[0]
        roi = cv2.copyMakeBorder(roi, diff//2, diff//2,0 ,0 ,cv2.BORDER_CONSTANT, value=255)
    roi = cv2.copyMakeBorder(roi, 0, 0, diff//2, diff//2,cv2.BORDER_CONSTANT, value=255)
    roi = cv2.copyMakeBorder(roi, index//10,index//10,index//10,index//10,cv2.BORDER_CONSTANT, value=255)
    # cv2.imshow("Image",roi)
    # cv2.waitKey(5000)
    roi = np.asarray(roi,dtype = np.float32)
    roi = TF.to_pil_image(roi)
    roi = transform_test(roi)
    roi = roi.view(1,1,64,64)
    symbols.append(model(roi.cuda()))
# cv2.destroyAllWindows()

stringexp = ""
for val in symbols:
    idx = torch.max(val,1)[1]
    to_calculate.append(classes[idx])
    stringexp += classes[idx]

print(stringexp)
ans = eval(stringexp)
print(ans)